In [1]:
import cv2
import os

In [58]:

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("/home/minh/Documents/project/face_recognition_pca_and_hog/haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
         
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
     
    cap = cv2.VideoCapture(0)
    id =1
    img_id = 250
     
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            file_name_path = "/home/minh/Documents/project/face_recognition_pca_and_hog/dataset/Ha."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
             
            cv2.imshow("Cropped face", face)
             
        if cv2.waitKey(1)==13 or int(img_id)==500: #13 is the ASCII character of Enter
            break
             
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
generate_dataset()


<>:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
/tmp/ipykernel_4744/3871446233.py:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


KeyboardInterrupt: 

In [1]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

In [2]:

folder='/home/minh/Documents/project/face_recognition_pca_and_hog/dataset' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image 

    img=img.resize([32,64])  # resize image  to a fixed size
    
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, channel_axis=-1) # obtain HOG features
    
    X.append(fd)

    # obtain training labels from file name
    
    if re.match("Ha*", filename):
      Yt=0
    else:
      Yt=1

    Y.append(Yt)
    
    i=i+1


In [3]:
fd.shape

(3780,)

In [4]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:

n_components = 25
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [6]:
# project training and test data to lower dimension 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
     

In [7]:

import pickle as pk
pk.dump(pca, open("c1_PCA.pkl","wb"))

In [8]:

print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0001, probability=True)


In [9]:
import joblib
joblib.dump(clf.best_estimator_, 'c2_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_svm_classifier.pkl']

In [10]:
y_pred = clf.predict(X_test_pca)
y_pred

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0])

In [11]:

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       159
           1       1.00      1.00      1.00        45

    accuracy                           1.00       204
   macro avg       1.00      1.00      1.00       204
weighted avg       1.00      1.00      1.00       204

